# Import Libraries

In [28]:
from random import *
import numpy as np
from multiprocessing import Pool
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numba
from numba import jit
import time
%matplotlib notebook 

### Parameters

In [29]:
probTree = 0.8
probBurning = 0.01
probImmune = 0.3
probLightning = 0.001

### Intializing the Universe (Forest)

In [30]:
@jit(forceobj=True, parallel=True)
#@numba.jit(nopython=True, parallel=True)
def InitializeUniverse(probTree = 0.8, probBurning = 0.01, arraySize = 100):
    
    startTime = time.time()
    
    #declare the dimensional array populated with zeros
    universe = np.zeros([arraySize, arraySize], dtype = int)
    
    #loop through the 2d array and generate a random number to cheeck if there is a tree and if the tree is burning or not
    #for i in numba.prange(len(universe)):
    for i in range(len(universe)):
        for j in range(len(universe[i])):
        #for j in numba.prange(len(universe[i])):
            
            #generate a random number
            probTreeAvailable = randrange(0, 100)
            probTreeAvailable = probTreeAvailable/100
            round(probTreeAvailable, 1)
            
            if(probTreeAvailable < probTree):
                #generate a random number
                probTreeBurning = randrange(0, 100)
                probTreeBurning = probTreeBurning/100
                round(probTreeBurning, 1)
            
                if(probTreeBurning < probBurning):
                      universe[i][j] = 2 #burning tree
            else:
                universe[i][j] = 1 #non-burning tree
    
    
    extendedUniverse = ExtendUniverse(universe) 
   
    
    if len(extendedUniverse):
        simulation = simulate(extendedUniverse)
        
    print("--- %s seconds - Execution Time (No Parallelisation)---" % (time.time() - startTime))
    return universe


### Extend the Universe (Forest)
The boundary of the Universe (forest) is extended using the periodic boundary conditons

In [31]:
def ExtendUniverse(universe):
    #gets the dimenson of the universe to be extended
    n = universe.shape[0]
    #extends an empty forest (array filled with zeros) by padding the grids by 2
    extendedUniverse = np.zeros((n+2, n+2), dtype=int)
    
    #this copies the universe grids into empty grids
    extendedUniverse[1:n+1, 1:n+1] = universe
    
    #this copies the values of the last row and assigns it to the first row (this reflects periodic boundary conditions)
    extendedUniverse[0, :] = extendedUniverse[n, :]
    
    #this copies the values of the second row and assigns it to the last row (this reflects periodic boundary conditions)
    extendedUniverse[n+1, :] = extendedUniverse[1, :]
     
    #this copies the values of the last column and assigns it to the first column
    extendedUniverse[:, 0] = extendedUniverse[:, n]
    
     #this copies the values of the second column and assigns it to the last column
    extendedUniverse[:, n+1] = extendedUniverse[:, 1]
    
    return extendedUniverse



### Fire Spread (Von Neumann Neighborhood Algorithm)
The spread of fire transition algorithm in this function was implemented based on Von Neumann Neighborhood

In [32]:
def FireSpread(i, j, extendedUniverse):
    cellValue = extendedUniverse[i][j]
    
    #checks if the cell value is empty(i.e. if the cell has no tree)
    if(cellValue == 0):
        #returns an empty cell i.e. 0
        extendedUniverse[i][j] = 0
        
    #checks if the site has a tree i.e. non-burning tree
    elif(cellValue == 1):
        #generate a random number for probability of the site immune from catching fire
        probSiteImmune = randrange(0, 100)
        probSiteImmune = probSiteImmune/100
        round(probSiteImmune, 1)
        
        #checks if the site is immuned from catching fire
        if (probSiteImmune < probImmune):
            extendedUniverse[i][j] = 1

        else:
             for di in range(-1, 2):
                for dj in range(-1, 2):
                    #try get the neighbouring value of the current cell
                    if abs(di) + abs(dj) == 1 and extendedUniverse[i+di, j+dj] == 2:
                        
                        #generate a random number for probability of the site lightning
                        probSitelightning = randrange(0, 100)
                        probSitelightning = probSitelightning/100
                        round(probSitelightning, 1)
                        
                        if (probSitelightning < probLightning):
                            # returns 2 i.e. lightning strikes the site
                            extendedUniverse[i][j] = 2
                        else:
                            # returns 2 i.e. the site catches fire
                            extendedUniverse[i][j] = 2
        
        extendedUniverse[i][j] = 1
        
    #checks if the site is burning already
    elif(cellValue == 2):
        #returns 0 i.e. the site is empty (burnt down)
        extendedUniverse[i][j] = 0
    
    #visualize(extendedUniverse)
   
    return extendedUniverse
    


### Simulate 

In [33]:
def simulate(extendedUniverse):
    n = extendedUniverse.shape[0]
    pool = Pool()
    
    
    results = []
    for i in range(1, n-1):
        for j in range(1, n-1):
            #maps the extendedUniverse into the FireSpread Function
            results.append(pool.apply(FireSpreadBasedOnMooreNeighborhood, (i, j, extendedUniverse)))
            #visualize(results)
    
   
    pool.close()
    pool.join()
    
    res = np.array(results)
   
    
    return res
    
    #return results
    
   
    

### Visualize the Forest Grid

In [34]:
def visualize(grid):
   
    # Define the color map
    cmap = plt.cm.colors.ListedColormap(['white', 'green', 'red'])
    # Define the bounds for the colors
    bounds = [0, 1, 2, 3]
    # Define the color for each value
    colors = ['white', 'green', 'red']
    # Create a color map object
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    # Create a figure and axis object
    fig, ax = plt.subplots()
    # Plot the grid
    ax.imshow(grid, cmap=cmap, norm=norm)
    # Remove the axis ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    # Set the title of the plot
    ax.set_title('Spread of Fire')
    # Create a color bar for the plot
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, ticks=bounds, boundaries=bounds)
    #cbar.ax.set_yticklabels(colors)
    
    
    def update(frame):
        ax.clear()
        # Update the grid
        ax.imshow(grid, cmap=cmap, norm=norm)
        return ax

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=grid, interval=500, blit=True)
    anim.save("Animate_Firespread.mp4")
    
    # Show the animation
    plt.show()


### Fire Spread (Moore Neighborhood Algortihm)

In [35]:
def FireSpreadBasedOnMooreNeighborhood(i, j, extendedUniverse):
    cellValue = extendedUniverse[i][j]
    
    #checks if the cell value is empty(i.e. if the cell has no tree)
    if(cellValue == 0):
        #returns an empty cell i.e. 0
        extendedUniverse[i][j] = 0
        
    #checks if the site has a tree i.e. non-burning tree
    elif(cellValue == 1):
        #generate a random number for probability of the site immune from catching fire
        probSiteImmune = randrange(0, 100)
        probSiteImmune = probSiteImmune/100
        round(probSiteImmune, 1)
        
        #checks if the site is immuned from catching fire
        if (probSiteImmune < probImmune):
            extendedUniverse[i][j] = 1

        else:
             for di in range(-1, 2):
                for dj in range(-1, 2):
                    #try get the neighbouring value of the current cell based on moore neighborhood
                    if i+di >= 0 and i+di < extendedUniverse.shape[0] and j+dj >= 0 and j+dj < extendedUniverse.shape[1]:
                     # check if the neighboring cell is not the current cell and is equal to 2
                     if (di != 0 or dj != 0) and extendedUniverse[i+di, j+dj] == 2:

                        #generate a random number for probability of the site lightning
                        probSitelightning = randrange(0, 100)
                        probSitelightning = probSitelightning/100
                        round(probSitelightning, 1)

                        if (probSitelightning < probLightning):
                            # returns 2 i.e. lightning strikes the site
                            extendedUniverse[i][j] = 2
                        else:
                            # returns 2 i.e. the site catches fire
                            extendedUniverse[i][j] = 2

        extendedUniverse[i][j] = 1
        
    #checks if the site is burning already
    elif(cellValue == 2):
        #returns 0 i.e. the site is empty (burnt down)
        extendedUniverse[i][j] = 0
    
    return extendedUniverse


In [36]:
#Entry Point for this simulation
InitializeUniverse()

--- 12.48971152305603 seconds - Execution Time (No Parallelisation)---


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])